# Stanford CoreNLP Coreference Resolution

### Before running this script
Run requirements.sh before running this script as it requires the coreNLP server to be up and running.

In [1]:
import jsonrpclib
from simplejson import loads
from nltk import sent_tokenize,word_tokenize

In [17]:
server = jsonrpclib.Server("http://localhost:8080")

In [19]:
string = "Barack Obama is the president of the USA. He will leave the office this year."
result = loads(server.parse(string))

In [20]:
print string
print result['coref']

Barack Obama is the president of the USA. He will leave the office this year.
[[[[u'the president of the USA', 0, 4, 3, 8], [u'Barack Obama', 0, 1, 0, 2]], [[u'He', 1, 0, 0, 1], [u'Barack Obama', 0, 1, 0, 2]]]]


In [34]:
import wikipedia as wk
batman = wk.page('Batman')
print type(batman.content)

<type 'unicode'>


In [35]:
sents = sent_tokenize(batman.content)

In [36]:
string = ''
for i in sents[0:10]:
    string += i
print string

Batman is a fictional superhero appearing in American comic books published by DC Comics.The character was created by artist Bob Kane and writer Bill Finger, and first appeared in Detective Comics #27 (May 1939).Originally named the "Bat-Man", the character is also referred to by such epithets as the Caped Crusader, the Dark Knight, and the World's Greatest Detective.Batman's secret identity is Bruce Wayne, an American billionaire, playboy, philanthropist, and owner of Wayne Enterprises.After witnessing the murder of his parents Thomas Wayne and Martha Wayne as a child, he swore revenge on criminals, an oath tempered by a sense of justice.Wayne trains himself physically and intellectually and crafts a bat-inspired persona to fight crime.Batman operates in the fictional Gotham City, with assistance from various supporting characters, including his butler Alfred, police commissioner Jim Gordon, and vigilante allies such as Robin.Unlike most superheroes, Batman does not possess any superp

In [25]:
result = loads(server.parse(string))
coreferences = result['coref']

In [26]:
for chunk in coreferences:
    print "***********************************"
    for text in chunk:
        print '{} is referenced to {}'.format(str(text[0][0]),str(text[1][0]))

***********************************
Batman is referenced to Batman 's
a fictional superhero appearing in American comic books published by DC Comics.The character is referenced to Batman 's
Batman is referenced to Batman 's
his is referenced to Batman 's
Batman is referenced to Batman 's
***********************************
the character is referenced to DC Comics.The character
***********************************
he is referenced to the `` Bat-Man ''
his is referenced to the `` Bat-Man ''
his is referenced to the `` Bat-Man ''
his is referenced to the `` Bat-Man ''
***********************************
the Caped Crusader , the Dark Knight , and the World 's Greatest Detective.Batman 's secret identity is referenced to Martha Wayne as a child
Bruce Wayne , an American billionaire , playboy , philanthropist , and owner of Wayne Enterprises.After witnessing the murder of his parents Thomas Wayne and Martha Wayne as a child , he swore revenge on criminals , an oath tempered by a sense of just

In [38]:
sents = string.split('.')
for i in sents[0:]:
    for j in sents[1:]:
        string = i+'.'+j
        print string
        result = loads(server.parse(string))
        try:
            coreferences = result['coref']
        except:
            print 'Coreference doesnt exists'
            continue
        print coreferences
        print '**************************************************'

Batman is a fictional superhero appearing in American comic books published by DC Comics.The character was created by artist Bob Kane and writer Bill Finger, and first appeared in Detective Comics #27 (May 1939)
Coreference doesnt exists
Batman is a fictional superhero appearing in American comic books published by DC Comics.Originally named the "Bat-Man", the character is also referred to by such epithets as the Caped Crusader, the Dark Knight, and the World's Greatest Detective
[[[[u"a fictional superhero appearing in American comic books published by DC Comics.Originally named the `` Bat-Man '' , the character is also referred to by such epithets as the Caped Crusader , the Dark Knight , and the World 's Greatest Detective", 0, 4, 2, 44], [u'Batman', 0, 0, 0, 1]]], [[[u'the character', 0, 21, 20, 22], [u"the `` Bat-Man ''", 0, 17, 15, 19]]]]
**************************************************
Batman is a fictional superhero appearing in American comic books published by DC Comics.Bat

### Idea behind Coreference Resolution
We can compare each string with the every other string in the text and if there exists a coreference between those two strings we can cosntruct an edge between the two strings (or decrease the weight of the edge by some lambda). If two sentences have some coreference between them then they are closer to each other as can be depicted by the small weight between the nodes pertaining to those sentences.
#### Challenges:
1. Many of the sentences can have coreference between them thus many of the edges will get their weights minimized in general won't affect the behaviour of algorithm on the graph.
2. Time complexity will increase further as the time taken to find coreference is O(n-squared) (considering parsing time to be constant)